In [2]:
# Constants
NANO_TO_MILLI = 1_000_000
NANO_TO_MICRO = 1_000
minLength = 2
maxLength = 10
testCount = 100

# Imports
import time
import string
import random
import tracemalloc
from prettytable import PrettyTable
import plotly as plt
from algo import *

# Тесты функций

In [ ]:
testTable = PrettyTable()
testTable.field_names = ["№", "Ожидаемое расстояние", "Ожидаемое Дамерау",  "Расстояние рекурсия", "Расстояние Кэш", "Дамерау Кэш"]
tests = [
    ("дмитрий", "андрей", 5, 5),
    ("река", "мука", 2, 2),
    ("1234", "2143", 3, 2)
    ]

for i in range(len(tests)):
    t = tests[i]
    testTable.add_row([i + 1, t[2], t[3], RecursiveLevenshtein(t[0], t[1]), CacheLevenshtein(t[0], t[1]), CacheDamerauLevenshtein(t[0], t[1])])

print(testTable)

# Тестирование по времени

In [ ]:
random.seed(time.time())

recursiveTests = {}
cacheTests = {}

for i in range(minLength, maxLength + 1):
    s1 = ''.join(random.choices(string.ascii_letters, k=i))
    s2 = ''.join(random.choices(string.ascii_letters, k=i))
    start = time.process_time_ns()
    for _ in range(testCount):
        _ = RecursiveLevenshtein(s1, s2)
    end = time.process_time_ns()
    recursiveTests[i] = (end - start) // i


    start = time.process_time_ns()
    for _ in range(testCount):
        _ = CacheLevenshtein(s1, s2)
    end = time.process_time_ns()
    cacheTests[i] = (end - start) // i

timeTable = PrettyTable()
timeTable.field_names = ["Длина строки", "Рекурсивный", "С кешированием"]
for i in range(minLength, maxLength + 1):
    timeTable.add_row([i, str(recursiveTests[i] // NANO_TO_MICRO) + "мкс", str(cacheTests[i] // NANO_TO_MICRO) + "мкс"])

print(timeTable)


graph = plt.graph_objs.Figure()
graph.add_scatter(x=list(range(minLength, maxLength + 1)), y=[recursiveTests[i] / NANO_TO_MICRO for i in recursiveTests], name="Рекурсивная реализация")
graph.add_scatter(x=list(range(minLength, maxLength + 1)), y=[cacheTests[i] / NANO_TO_MICRO for i in cacheTests], name="Реализация с кешэм")
graph.update_layout(xaxis_title="Длина строки", yaxis_title="Время вычисления, мкс")
graph.show()

# Тестирование по памяти

In [ ]:
testCount = 10
recursiveMemory = {}
cacheMemory = {}
# Memory
for i in range(minLength, maxLength + 1):
    # рекурсивная реализация зависит от строки, так как строит дерево рекурсивных вызовов
    recursiveMemory[i] = 0
    for _ in range(testCount):
        s1 = ''.join(random.choices(string.ascii_letters, k=i))
        s2 = ''.join(random.choices(string.ascii_letters, k=i))
        tracemalloc.start()
        _ = RecursiveLevenshtein(s1, s2)
        recursiveMemory[i] += tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
    recursiveMemory[i] //= testCount

    # Кэш всегда одинаковый по памяти, не зависимо от строки
    tracemalloc.start()
    _ = CacheLevenshtein(s1, s2)
    cacheMemory[i] = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()


memTable = PrettyTable()
memTable.field_names = ["Длина строки", "Рекурсивный", "С кешированием"]
for i in range(minLength, maxLength + 1):
    memTable.add_row([i, str(recursiveMemory[i]) + " байт", str(cacheMemory[i]) + " байт"])

print(memTable)


graph = plt.graph_objs.Figure()
graph.add_scatter(x=list(range(minLength, maxLength + 1)), y=[recursiveMemory[i] for i in recursiveMemory], name="Рекурсивная реализация")
graph.add_scatter(x=list(range(minLength, maxLength + 1)), y=[cacheMemory[i] for i in cacheMemory], name="Реализация с кешэм")
graph.update_layout(xaxis_title="Длина строки", yaxis_title="Пиковая память, байт")
graph.show()